# Notebook

## Definitions

- **voxel**: _**vo**lumetric pi**xel**_ smallest simulated object
- **block**: defined for comparing with Minecraft purposes. Its dimensions are $16 \cdot 16 \cdot 16 $ voxels
- **chunk**: smallest loadable from memory world data. Its dimensions are $256 \cdot 256 \cdot 256$ voxels, or $16 \cdot 16 \cdot 16$ blocks
- **cluster**: smallest saveable on disc world data. Its dimensions are $16 \cdot 16 \cdot 16$ chunks

In [2]:
block = 16 * 16 * 16 # voxels
chunk = block * (16 * 16 * 16)
cluster = chunk * (16 * 16 * 16)

print( f"{block = :_}" )
print( f"{chunk = :_}" )
print( f"{cluster = :_}" )

block = 4_096
chunk = 16_777_216
cluster = 68_719_476_736


Every voxel should know its color and material (in comparsion with Minecraft -- block owning it). At this moment, let's assume $2^{32}$ materials.

In [25]:
voxel_info_size = { # bits
  'block_type': 32,
  'color': 8 + 8 + 8 + 8
}

info_to_store_sum = sum( voxel_info_size.values() )

print( f"{voxel_info_size = }" )
print( f"{info_to_store_sum = :_}" )

voxel_info_size = {'block_type': 32, 'color': 32}
info_to_store_sum = 64


## Estimations

In [26]:
# helper functions

def number_bits( x:int ):
    return 1 if x == 0 else x.bit_length()

def next_power_of_two( x:int ):
    return 1 if x == 0 else 2 ** (x - 1).bit_length()

For the world to be diverse, it must be filled by different things. Let's assume below list of *things*

In [37]:
things = {
  'crops': 15,
  'flowers': 15,
  'trees': 5,
  'dirt-like': 10,
  'stones': 10,
  'fluids': 5,
  'others': 20,
}

things_count = sum( things.values() )

print( f"{things_count = :_}" )
print( f"things_power_of_two_count = {next_power_of_two( things_count ):_}" )

things_count = 80
things_power_of_two_count = 128


Bits needed to store `things_count` value, should be a minimal size of the voxel `block_type` property. This implies the following calculations

In [43]:
def print_estimations():
  types_per_cluster = next_power_of_two( things_count )

  print( f"{types_per_cluster = :_}" )
  print( f"types_per_cluster * voxels_in_cluster = {types_per_cluster * cluster:_}" )
  print( f"types_per_cluster * voxels_in_cluster [Gib] = {(types_per_cluster * cluster) / 1024 ** 3:_}" )
  print( f"types_per_cluster * voxels_in_cluster [GiB] = {(types_per_cluster * cluster) / 1024 ** 3 / 8:_}" )

print_estimations()

types_per_cluster = 128
types_per_cluster * voxels_in_cluster = 8_796_093_022_208
types_per_cluster * voxels_in_cluster [Gib] = 8_192.0
types_per_cluster * voxels_in_cluster [GiB] = 1_024.0


Above calculations means that, very basic optimisation requires ~1TB of memory space. This is absolutely unacceptable

## Comparsion

In [48]:
def print_minecraft_numbers():
  minecraft_chunk_side = 16
  minecraft_chunk = minecraft_chunk_side ** 2 * 320
  minecraft_simulation_distance_in_chunks = 16
  minecraft_simulation_blocks = minecraft_chunk * (minecraft_simulation_distance_in_chunks * 2) ** 2
  block_types_bits = number_bits( 1000 )

  print( f"{minecraft_chunk = :_}" )
  print( f"{minecraft_simulation_blocks = :_}" )
  print( f"{block_types_bits = :_}" )
  print( f"minecraft_simulation_blocks [MiB] = {(minecraft_simulation_blocks * block_types_bits) / 1024 ** 2 / 8:_}" )

print_minecraft_numbers()

minecraft_chunk = 81_920
minecraft_simulation_blocks = 83_886_080
block_types_bits = 10
minecraft_simulation_blocks [MiB] = 100.0
